In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import r_regression

In [ ]:
# load and prepare right and left striatal masks

mask = nib.load('/content/drive/MyDrive/1 Sagol Project/Mask/Tian_Subcortex_S3_3T.nii').get_fdata()

striatum = list(range(11, 19)) + list(range(22,24)) + list(range(36, 44)) + list(range(47,49))


bool_mask = np.isin(mask, striatum)

mask[~bool_mask] = 0
mask[bool_mask] = 1

r_mask, l_mask = np.copy(mask), np.copy(mask)
r_mask[45:,:,:] = 0;
r_mask = r_mask.flatten()
l_mask[:45,:,:] = 0;
l_mask = l_mask.flatten()

mask = mask.flatten()
mask = np.stack([mask for i in range(0,50)])
mask = np.transpose(mask)

In [ ]:
# initialize normalization operator

mm = MinMaxScaler()

In [ ]:
# establish data paths

fmri_folder_path = '/content/drive/MyDrive/1 Sagol Project/Features/'
pet_folder_path = '/content/drive/MyDrive/1 Sagol Project/PET/'

In [ ]:
# load PET data

original_Y = []

for file in os.listdir(pet_folder_path):
  file_path = os.path.join(pet_folder_path, file)
  temp_pet = nib.load(file_path).get_fdata()
  temp_pet = temp_pet.flatten()[mask[:,0] != 0]
  temp_pet = mm.fit_transform(temp_pet.reshape(-1,1)).flatten()
  original_Y.append(temp_pet)


In [ ]:
# load FC data

original_X = []

for file in os.listdir(fmri_folder_path):
  file_path = os.path.join(fmri_folder_path, file)
  temp_fmri = nib.load(file_path).get_fdata()
  temp_fmri = temp_fmri.reshape(-1, temp_fmri.shape[-1])
  temp_fmri = temp_fmri[~np.all(mask == 0, axis=1)]
  temp_fmri = mm.fit_transform(temp_fmri)
  original_X.append(temp_fmri)

In [ ]:
# select only subjects which have consistent imaging and clinical lateralization labels

X, Y = [], []

for i in range(26):
  if i not in (4,5,11,15,17,20,22,24):
    Y.append(original_Y[i])
    X.append(original_X[i])

In [ ]:
# 0 - right, 1 - left, 18 Subjects
clinical_labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]

In [ ]:
comp = [{} for i in range(18)]
pred_list = [0 for i in range(18)]

In [ ]:
# LOO-CV classification routine

for i in range(18):

  features = np.concatenate(X[:17])
  target = np.concatenate(Y[:17])
  xgb_model = xgb.XGBRegressor(max_depth = 3).fit(features, target)
  pred = xgb_model.predict(X[17])

  pred_copy = np.copy(pred)
  unmasked_pet = np.copy(mask[:,0])

  for k in range(len(mask[:,0])):
    if unmasked_pet[k] == 1:
      unmasked_pet[k] = pred_copy[0];
      pred_copy = pred_copy[1:];

  pred = unmasked_pet

  # calculating mean PET activation in each hemisphere

  pred_SUVR_r, pred_SUVR_l = pred[r_mask!= 0].mean(), pred[l_mask!= 0].mean()
  pred_label = (pred_SUVR_r > pred_SUVR_l).astype(int)
  comp[17-i] = {'Predicted': pred_label, 'Real': clinical_labels[17-i], 'Right_Prediction': pred_label == clinical_labels[17-i]}
  pred_list[17-i] = pred_label

  X.insert(0, X.pop(17))
  Y.insert(0, Y.pop(17))

In [ ]:
right_pred = sum([comp[i]['Right_Prediction'] for i in range(18)])

accuracy = right_pred/18
[accuracy, right_pred]

[0.6111111111111112, 11]

In [ ]:
comp

[{'Predicted': 1, 'Real': 1, 'Right_Prediction': True},
 {'Predicted': 1, 'Real': 1, 'Right_Prediction': True},
 {'Predicted': 1, 'Real': 1, 'Right_Prediction': True},
 {'Predicted': 1, 'Real': 1, 'Right_Prediction': True},
 {'Predicted': 1, 'Real': 1, 'Right_Prediction': True},
 {'Predicted': 0, 'Real': 0, 'Right_Prediction': True},
 {'Predicted': 1, 'Real': 0, 'Right_Prediction': False},
 {'Predicted': 1, 'Real': 0, 'Right_Prediction': False},
 {'Predicted': 1, 'Real': 0, 'Right_Prediction': False},
 {'Predicted': 1, 'Real': 0, 'Right_Prediction': False},
 {'Predicted': 1, 'Real': 0, 'Right_Prediction': False},
 {'Predicted': 1, 'Real': 1, 'Right_Prediction': True},
 {'Predicted': 1, 'Real': 0, 'Right_Prediction': False},
 {'Predicted': 0, 'Real': 0, 'Right_Prediction': True},
 {'Predicted': 0, 'Real': 1, 'Right_Prediction': False},
 {'Predicted': 0, 'Real': 0, 'Right_Prediction': True},
 {'Predicted': 0, 'Real': 0, 'Right_Prediction': True},
 {'Predicted': 0, 'Real': 0, 'Right_Predi